# ARTICLE SCRAPING

In [17]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
#Set the executable path for scraping

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\aecm0\.wdm\drivers\chromedriver\win32\92.0.4515.107]


In [3]:
# Visit the mars nasa news site

url = 'https://redplanetscience.com'
browser.visit(url)

# Optional delay for loading the page
# Con esta línea se están buscando elementos con una combinación específica de (div) y el atributo (list_text)
# Con wait_time, se le dice al browser que espere un segundo antes de buscar los componentes
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
#HTML parser

html = browser.html
news_soup = soup(html, 'html.parser')
# Slide_elem is the variable to look for the tag <div/>
# list_text  is the class we are looking for
# Select_one se utiliza para que regrese el primer item que hace match
slide_elem = news_soup.select_one('div.list_text')

In [5]:
# Se está añadiendo .find para especificar qué dato se quiere encontrar, que en este caso es el "content_title"
slide_elem.find('div', class_='content_title')

<div class="content_title">InSight's 'Mole' Team Peers into the Pit</div>

In [6]:
#Para obtener sólo el texto del resultado anterior con el .get_text
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"InSight's 'Mole' Team Peers into the Pit"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Efforts to save the heat probe continue.'

# IMAGE SCRAPING

### Featured images

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button

full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [11]:
#Para obterner siempre la imagen más reciente se usa la etiqueta <img/> y la clase fancybox-img
# Find the relative image url
#.get "src" pulls the link to the image
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [12]:
#Para tener la URL completa y ver la imagen se debe completarla
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [14]:
#Se va a crear un dataframe desde la tabla HTML
#La función read_html() específicamente busca y devuelve la lista de tablas encontradas en un HTML
#Con el índice [0] se específica que se quiere obtener la primera tabla que se encuentre en la lista de tablas

df = pd.read_html('https://galaxyfacts-mars.com')[0]

#Se añaden las columnas al dataframe
df.columns=['description', 'Mars', 'Earth']

#Se define a la columna "descripción" cómo el índice de la tabla
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [16]:
#Para cerrar la sesión automática del browser, que sólo queremos tener abierto mientras se hace el scraping:

browser.quit()